## mpisppy setup

Setting up pyomo's extension mpisppy for stochastic programming can become time consuming for various reasons. This post is meant as a copy and paste summary to install mpi-sppy including a first trouble shooting. 
Moreover we include tests to assure that the installation went well.

Following [pyomo read the docs](https://mpi-sppy.readthedocs.io/en/latest/install_mpi.html) the setup can be straight forward by just running the following comands in this order:

1. `conda install openmpi`
    - alternative `conda install -c anaconda openmpi`
1. `conda install mpi4py`
1. `pip install mpi-sppy`

### prerequisites

1. `conda install -c conda-forge pyomo`
1. `conda install -c conda-forge glpk`

## trouble shooting

### Problems installing openmpi

c&p walkthrough to install openmpi (on Ubutu 20.04)[^install_open_mpi]
**remark:** windows users may need browse for an alternative solution, dropword look for alternative to gcc

1. check if installed `mpiexec --version`
1. update package repository `sudo apt-get update`
1. check gcc compiler and libs
    - check: `gcc --version` and `g++ --version`
    - install:`sudo apt install build-essential`
1. install openmpi `sudo apt-get install openmpi-bin openmpi-doc libopenmpi-dev`


[^install_open_mpi]: reference https://webpages.charlotte.edu/abw/coit-grid01.uncc.edu/ParallelProgSoftware/Software/OpenMPIInstall.pdf , alternative : https://edu.itp.phys.ethz.ch/hs12/programming_techniques/openmpi.pdf

## Test mpisppy installation

We use the following farmer example from [pyomo read the docs](https://mpi-sppy.readthedocs.io/en/latest/examples.html) to test the installation.

For simplicity we only test here the solution gained from the extensive form, but give some basic tests in case you have cplex installed.
The extensive from of a stochastic program is simply said a reformulation of the stochastic programm as a big linear program. 

In [ ]:
# additional libs
import math

In [ ]:
# test deterministic
import pyomo.environ as pyo

def build_model(yields):
    model = pyo.ConcreteModel()

    # Variables
    model.X = pyo.Var(["WHEAT", "CORN", "BEETS"], within=pyo.NonNegativeReals)
    model.Y = pyo.Var(["WHEAT", "CORN"], within=pyo.NonNegativeReals)
    model.W = pyo.Var(
        ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"],
        within=pyo.NonNegativeReals,
    )

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    model.PURCHASE_COST = 238 * model.Y["WHEAT"] + 210 * model.Y["CORN"]
    model.SALES_REVENUE = (
        170 * model.W["WHEAT"] + 150 * model.W["CORN"]
        + 36 * model.W["BEETS_FAVORABLE"] + 10 * model.W["BEETS_UNFAVORABLE"]
    )
    model.OBJ = pyo.Objective(
        expr=model.PLANTING_COST + model.PURCHASE_COST - model.SALES_REVENUE,
        sense=pyo.minimize
    )

    # Constraints
    model.CONSTR= pyo.ConstraintList()

    model.CONSTR.add(pyo.summation(model.X) <= 500)
    model.CONSTR.add(
        yields[0] * model.X["WHEAT"] + model.Y["WHEAT"] - model.W["WHEAT"] >= 200
    )
    model.CONSTR.add(
        yields[1] * model.X["CORN"] + model.Y["CORN"] - model.W["CORN"] >= 240
    )
    model.CONSTR.add(
        yields[2] * model.X["BEETS"] - model.W["BEETS_FAVORABLE"] - model.W["BEETS_UNFAVORABLE"] >= 0
    )
    model.W["BEETS_FAVORABLE"].setub(6000)

    return model

yields = [2.5, 3, 20]
model = build_model(yields)
solver = pyo.SolverFactory("glpk")
solver.solve(model)

# Display the objective value to one decimal place
assert math.isclose(pyo.value(model.OBJ), -118600.0, 
                    rel_tol = 1e-09, abs_tol = 1e-09),'wrong objective value in deterministic solution, check prerequisites'

In [ ]:
# test extensive form
import mpisppy.utils.sputils as sputils
from mpisppy.opt.ef import ExtensiveForm

def scenario_creator(scenario_name):
    if scenario_name == "good":
        yields = [3, 3.6, 24]
    elif scenario_name == "average":
        yields = [2.5, 3, 20]
    elif scenario_name == "bad":
        yields = [2, 2.4, 16]
    else:
        raise ValueError("Unrecognized scenario name")

    model = build_model(yields)
    sputils.attach_root_node(model, model.PLANTING_COST, [model.X])
    model._mpisppy_probability = 1.0 / 3
    return model


options = {"solver": "glpk"}
all_scenario_names = ["good", "average", "bad"]
ef = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef.solve_extensive_form()

objval = ef.get_objective_value()
assert math.isclose(objval, -108390.0, 
                    rel_tol = 1e-09, abs_tol = 1e-09),'wrong objective value for extensive form solution, check mpi-sppy setup'

[    0.00] Initializing mpi-sppy
[    0.00] Initializing SPBase
